In [2]:
!pip install python-sat[pblib,aiger]

     |████████████████████████████████| 1.8 MB 8.0 MB/s 
     |████████████████████████████████| 3.4 MB 44.1 MB/s 
     |████████████████████████████████| 45 kB 2.8 MB/s 
  Created wheel for parsimonious: filename=parsimonious-0.8.1-py3-none-any.whl size=42723 sha256=96ec0cce10db4d259ce2abcaf7d4d862f44c0011a23977902c103bdb7055c33e
  Stored in directory: /root/.cache/pip/wheels/88/5d/ba/f27d8af07306b65ee44f9d3f9cadea1db749a421a6db8a99bf
Successfully built parsimonious


In [ ]:
from itertools import chain, combinations, permutations
import pysat.solvers
from pysat.card import CardEnc as encode
from tqdm import tqdm
from random import sample

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(1, len(s)+1))

def check_suppps(s):
    N = len(set(s))
    s = ''.join(map(str, s))
    for i in powerset(range(1,N+1)):
        if not any(''.join(map(str, p)) in s for p in permutations(i)):
            return False
    return True

L = 73 

def pow_with_rest():
    s = [ i for i in list(powerset(range(1, 8 + 1))) if 3 <= len(i) <= 3]
    return sample(s, len(s))

print(pow_with_rest())

def generate_cover(tiles, L):
    out = []
   
    for i in tiles:
        temp = []
        for p in permutations(i):
            for j in range(L-len(i)+1):
                temp.append( tuple([0]*(L-len(i) - j)) + p + tuple([0]*j))
        #out.append(sample(temp, 7))
        out.append(temp)

    print('tiles ', sum(len(i) for i in out))
    return out

def generate_clauses(tiles):
    N = len(tiles)
    print(N)
    clauses = []
    t = 0
    d = dict()
    for i in range(N):
        for j in range(len(tiles[i])):
            t+=1
            d[(i, j)] = t
    
    print(t)

    solver = pysat.solvers.Cadical()

    for i in tqdm(range(N)):
        #clauses.append([d[(i, k)] for  k in range(len(tiles[i]))])
        solver.add_clause([d[(i, k)] for  k in range(len(tiles[i]))])

        for j in  range(len(tiles[i])):
            t = tiles[i][j]
            g = d[(i,j)]

            for di in range(N):
                for dj in range(len(tiles[di])):  #(di == i and not dj == j ) or
                    if  any( (tiles[di][dj][k] != 0 and t[k] !=0) and tiles[di][dj][k] != t[k] for k in range(L)):

                        #clauses.append([-g, -d[di, dj]])
                        solver.add_clause([-g, -d[di, dj]])
    

    
    # for clause in tqdm(clauses):
    #     solver.add_clause(clause)
    print('aaaa')

    sat = solver.solve()

    if sat:
        s = set()
        for m in solver.enum_models():
            
            
            model = [i for i in m if i>0]

            out = []
            t=0
            for i in range(N):
                for j in range(len(tiles[i])):
                    t+=1
                    if t in model:
                        out.append(tiles[i][j])
            
            supps = [0]*L
            for  i in out:
                for k, v in enumerate(i):
                    if v != 0:
                        supps[k] = v

            if tuple(supps) not in s:
                print(supps)
                print(check_suppps(supps))
                s.add(tuple(supps))

        
    else:
        print('unsat')











cov = generate_cover(pow_with_rest(), L )
# print(sum(len(i) for i in cov))

generate_clauses(cov)
# for i in m:
#     print(i)

# [5, 3, 1, 8, 7, 1, 3, 5, 7, 8, 4, 5, 3, 8, 2, 5, 2, 1, 6, 2, 5, 3, 2, 7, 2, 6, 8, 1, 2, 3, 4, 8, 4, 2, 6, 3, 1, 4, 8, 6, 7, 5, 2, 1, 2, 7, 4, 5, 2, 4, 1, 7, 6, 4, 5, 4, 1, 6, 5, 7, 4, 3, 6, 8, 5, 1, 7, 5, 6, 3, 7, 8, 2]
# [5, 3, 1, 8, 7, 1, 3, 5, 7, 8, 4, 5, 3, 8, 2, 5, 2, 1, 6, 2, 5, 3, 2, 7, 2, 6, 8, 1, 2, 3, 4, 8, 4, 2, 6, 3, 1, 4, 8, 6, 7, 2, 5, 1, 2, 7, 4, 5, 2, 4, 1, 7, 6, 4, 5, 4, 1, 6, 5, 7, 4, 3, 6, 8, 5, 1, 7, 5, 6, 3, 7, 8, 2]
# [5, 3, 1, 8, 7, 1, 4, 6, 2, 8, 4, 5, 3, 8, 2, 5, 2, 1, 3, 7, 5, 3, 2, 7, 5, 6, 8, 1, 2, 3, 4, 8, 7, 2, 6, 3, 1, 4, 8, 6, 7, 3, 4, 1, 2, 7, 4, 5, 5, 4, 1, 7, 6, 4, 5, 2, 1, 6, 5, 2, 4, 3, 6, 8, 5, 1, 7, 5, 6, 3, 7, 8, 5]
# [5, 3, 1, 8, 7, 1, 4, 6, 2, 8, 4, 5, 3, 8, 2, 5, 2, 1, 3, 7, 5, 3, 2, 7, 5, 6, 8, 1, 2, 3, 4, 8, 7, 2, 6, 3, 1, 4, 8, 6, 7, 3, 4, 1, 2, 7, 4, 5, 1, 4, 1, 7, 6, 4, 5, 2, 1, 6, 5, 2, 4, 3, 6, 8, 5, 1, 7, 5, 6, 3, 7, 8, 5]
# [5, 3, 1, 8, 7, 1, 4, 6, 2, 8, 4, 5, 3, 8, 2, 5, 2, 1, 3, 7, 5, 3, 2, 7, 5, 6, 8, 1, 2, 3, 4, 8, 7, 2, 6, 3, 1, 4, 8, 6, 7, 3, 4, 1, 2, 7, 4, 5, 1, 4, 1, 7, 6, 4, 5, 2, 1, 6, 5, 2, 4, 3, 6, 8, 5, 1, 7, 5, 6, 3, 8, 7, 5]



[(4, 7, 8), (1, 4, 7), (3, 4, 5), (3, 4, 7), (1, 3, 4), (4, 5, 8), (1, 2, 3), (1, 5, 6), (2, 3, 5), (2, 4, 6), (3, 5, 6), (1, 6, 8), (4, 5, 6), (1, 5, 7), (1, 4, 5), (5, 6, 8), (1, 4, 8), (2, 7, 8), (1, 2, 8), (2, 3, 7), (2, 6, 8), (1, 6, 7), (6, 7, 8), (4, 6, 7), (2, 4, 8), (1, 3, 8), (1, 5, 8), (4, 6, 8), (1, 2, 7), (3, 5, 7), (1, 3, 6), (1, 2, 6), (1, 2, 5), (2, 6, 7), (3, 5, 8), (2, 4, 5), (2, 3, 4), (4, 5, 7), (3, 7, 8), (1, 4, 6), (2, 3, 6), (2, 5, 8), (3, 4, 8), (1, 2, 4), (3, 4, 6), (2, 5, 7), (3, 6, 7), (3, 6, 8), (2, 3, 8), (1, 7, 8), (2, 5, 6), (5, 7, 8), (1, 3, 7), (1, 3, 5), (2, 4, 7), (5, 6, 7)]
tiles  23856
56
23856


100%|██████████| 56/56 [1:42:18<00:00, 109.62s/it]


aaaa
[5, 3, 2, 4, 6, 1, 8, 8, 7, 3, 2, 7, 1, 5, 6, 8, 4, 3, 1, 2, 6, 5, 1, 3, 8, 6, 7, 2, 5, 1, 8, 7, 6, 1, 4, 5, 6, 7, 5, 3, 3, 1, 7, 4, 5, 3, 8, 2, 7, 4, 8, 2, 1, 4, 8, 5, 2, 4, 6, 3, 5, 8, 7, 3, 6, 1, 6, 4, 7, 3, 6, 2, 8]
False
[5, 3, 2, 4, 6, 1, 8, 8, 7, 3, 2, 7, 1, 5, 6, 8, 4, 3, 1, 2, 6, 5, 1, 3, 8, 6, 7, 2, 5, 1, 8, 7, 6, 1, 4, 5, 6, 7, 5, 3, 3, 1, 7, 4, 5, 3, 8, 2, 7, 4, 8, 2, 1, 4, 8, 5, 2, 4, 6, 3, 5, 8, 7, 0, 1, 3, 6, 4, 7, 3, 6, 2, 8]
False
[5, 3, 2, 4, 6, 1, 8, 8, 7, 3, 2, 7, 1, 5, 6, 8, 4, 3, 1, 2, 6, 5, 1, 3, 8, 6, 7, 2, 5, 1, 8, 7, 6, 1, 4, 5, 6, 7, 5, 3, 3, 1, 7, 4, 5, 3, 8, 2, 7, 4, 8, 2, 1, 4, 8, 5, 2, 4, 6, 3, 5, 8, 7, 2, 1, 3, 6, 4, 7, 3, 6, 2, 8]
False
[5, 3, 2, 4, 6, 1, 8, 8, 7, 3, 2, 7, 1, 5, 6, 8, 4, 3, 1, 2, 6, 5, 1, 3, 8, 6, 7, 2, 5, 1, 8, 7, 6, 1, 4, 5, 6, 7, 5, 3, 3, 1, 7, 4, 5, 3, 8, 2, 7, 4, 8, 2, 1, 4, 8, 5, 2, 4, 6, 3, 5, 8, 7, 4, 1, 3, 6, 4, 7, 3, 6, 2, 8]
False
[5, 3, 2, 4, 6, 1, 8, 8, 7, 3, 2, 7, 1, 5, 6, 8, 4, 3, 1, 2, 6, 5, 1, 3, 8, 6, 7, 2, 5, 1, 

In [ ]:
a = (1,2,0,0)
b = (0,0,1,1)
[a[k]!=0 and b[k] != a[k] for k in range(len(a))]
# any( tiles[di][dj][k] != 0 and tiles[di][dj][k] != t[k] for k in range(len(t))):

[True, True, False, False]